In [18]:
import requests
from bs4 import BeautifulSoup, Tag
import re


response = requests.get(
    "https://www.sigma-computer.com/search?search=laptop&submit_search=&route=product%2Fsearch"
)
records: list[tuple] = []
loop = 1
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    productTags: list[Tag] = soup.find_all(
        "div", attrs={"class": "product-item-container"}
    )
    for e in productTags:
        if "processor" in e.text.lower():
            link = e.find("div", attrs={"class": "caption"}).find("a").attrs["href"]
            productResponse = requests.get(f"https://www.sigma-computer.com/{link}")

            productsoup = BeautifulSoup(productResponse.content, "html.parser")
            productLink = f"https://www.sigma-computer.com/{link}"
            title = productsoup.find("div", attrs={"class": "title-product"})
            title = title.text if title != None else ""
            stock = (
                productsoup.find("span", attrs={"class", "status-stock"}).text
                if productsoup.find("span", attrs={"class", "status-stock"}) != None
                else "NO"
            )
            newPrice = productsoup.find("span", attrs={"class", "price-new"})
            newPrice = newPrice.text.split()[0] if newPrice != None else ""
            #brand = productsoup.find("div", attrs={"class", "brand"})
            #brand = brand.text.strip().split(":")[1] if brand != None else ""

            #model = productsoup.find("div", attrs={"class", "model"})
            #model = model.text.strip().split(":")[1] if model != None else ""
            #views = productsoup.find(string=lambda text: text and "Viewed : " in text)
            #views = views.find_parent("div") if views != None else None
            #views = (
             #   views.text.split(":")[1].replace("items", "").strip()
              #  if views != None
               # else ""
            #)
            specificationDic = dict()
            # get specification tab by regular expression and add it to dictionary
            div = productsoup.find("div", attrs={"class": "inner-box-desc"})
            divText = (
                div.text.strip().replace("\t", " ").replace("\r\n", "").replace("\"","")
                if div != None
                else ""
            )
            pattern = re.compile(r"(.+?):(.+)")
            matches = pattern.findall(divText)

            for header, detail in matches:
                if detail != "":
                    detail = detail.strip().replace("\n", "")
                    header = header.strip().replace("\n", "").lower()
                    specificationDic.update({header: detail})

            records.append(
                (
                    productLink.replace("\n", ""),
                    title.replace("\n", ""),
                    stock.replace("\n", ""),
                    newPrice.replace("\n", ""),
                    specificationDic,
                )
            )
            loop += 1
            print(loop)
            if loop == 50:
                break
#distinct keys to be able add them to file header
    distinct_keys_set: set = set()
    for rowTuple in records:
        for key in list(rowTuple)[-1].keys():
            distinct_keys_set.add(key)

    fileHeader = [
        "productLink",
        "title",
        "stock",
        "new-price",
        
    ]
    fileHeader.extend(distinct_keys_set)
  

    with open("Laptops.csv", mode="w", encoding="utf-8") as df:
        header = ",".join(fileHeader)
        df.write(f"{header}\n")
        fileRecord = ""
        for e in records:
            fileRecord = (
                f'"{e[0]}","{e[1]}","{e[2]}","{e[3]}"'
            )
            for x in distinct_keys_set:
                # Retrieve value with a default value if key doesn't exist e[-1].get(x,"")
                fileRecord += "," + "\"" + e[-1].get(x,"Unknown") + "\""

            df.write(fileRecord + "\n")

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [ ]:
import pandas as pd
df =pd.read_csv("Laptops.csv")
df


# Git Hub URL
https://github.com/AWasseem/Epsilon/tree/main/Session%2012%20datacollection

In [17]:
link = "https://www.sigma-computer.com/item?id=7262&name=Asus_ROG_Strix_G17_G713PV-HX019W_Gaming_Laptop_17.3-inch_FHD_144Hz_AMD_R9_7845HX_16GB_RAM_1TB_SSD_RTX_4060_8GB_Win11_+_ROG_Backpack,_90NR0C34-M00BL0"

productResponse = requests.get(link)

productsoup = BeautifulSoup(productResponse.content, "html.parser")

div = productsoup.find("div", attrs={"class": "inner-box-desc"})
divText = (
                div.text.strip().replace("\t", " ").replace("\r\n", "").replace("\"","")
                if div != None
                else ""
            )
print(divText)

Brand : ASUS



 
Model : G713PV-HX019W 
Seller : sigma 
Viewed :   3884 items 

The Asus ROG Strix G17 G713PV-HX019W is a powerful gaming laptop that combines high-end performance with excellent visuals. Here’s a breakdown of its key specs:

Specifications:


Display: 17.3-inch FHD (1920 x 1080) 144Hz

Great for smooth visuals during gameplay with minimal motion blur.



Processor: AMD Ryzen 9 7845HX

12 cores, 24 threads – Ideal for demanding games and multitasking.



Graphics: NVIDIA GeForce RTX 4060 (8GB GDDR6)

Handles modern AAA titles and content creation with ray tracing support.



RAM: 16GB DDR5

High-speed memory ensures fast loading times and smooth multitasking.



Storage: 1TB PCIe SSD

Plenty of space for games, files, and fast data access.



Operating System: Windows 11

Supports the latest features and optimizations for gaming.



Extras:

ROG Backpack: Practical for gamers on the go.
90NR0C34-M00BL0: Model number for tracking the exact version.




Ideal For:

Gamer